In [3]:
import vectorbt as vbt

price = vbt.YFData.download('BTC-USD').get('Close')

pf = vbt.Portfolio.from_holding(price, init_cash=100)
pf.total_profit()

5841.769798466174

In [4]:
fast_ma = vbt.MA.run(price, 10)
slow_ma = vbt.MA.run(price, 50)
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=100)
pf.total_profit()

15774.46164516048

In [5]:
import numpy as np

symbols = ["BTC-USD", "ETH-USD"]
price = vbt.YFData.download(symbols, missing_index='drop').get('Close')

n = np.random.randint(10, 101, size=1000).tolist()
pf = vbt.Portfolio.from_random_signals(price, n=n, init_cash=100, seed=42)

mean_expectancy = pf.trades.expectancy().groupby(['randnx_n', 'symbol']).mean()
fig = mean_expectancy.unstack().vbt.scatterplot(xaxis_title='randnx_n', yaxis_title='mean_expectancy')
fig.show()

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)
/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_data[c].loc[:, s] = col_data
/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_data[c].lo

In [18]:
symbols = ["BTC-USD", "ETH-USD", "LTC-USD"]
price = vbt.YFData.download(symbols, missing_index='drop').get('Close')
print(price)
price = price[price.index >= '2022-01-01']

windows = np.arange(2, 101)
fast_ma, slow_ma = vbt.MA.run_combs(price, window=windows, r=2, short_names=['fast', 'slow'])
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')
pf = vbt.Portfolio.from_signals(price, entries, exits, **pf_kwargs)

fig = pf.total_return().vbt.heatmap(
    x_level='fast_window', y_level='slow_window', slider_level='symbol', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: Fut

symbol                          BTC-USD      ETH-USD    LTC-USD
Date                                                           
2017-11-09 00:00:00+00:00   7143.580078   320.884003  64.269699
2017-11-10 00:00:00+00:00   6618.140137   299.252991  59.260101
2017-11-11 00:00:00+00:00   6357.600098   314.681000  62.303299
2017-11-12 00:00:00+00:00   5950.069824   307.907990  59.005402
2017-11-13 00:00:00+00:00   6559.490234   316.716003  61.396500
...                                 ...          ...        ...
2023-05-19 00:00:00+00:00  26890.128906  1812.589478  91.702385
2023-05-20 00:00:00+00:00  27129.585938  1820.478027  92.371315
2023-05-21 00:00:00+00:00  26753.826172  1804.531372  92.278870
2023-05-22 00:00:00+00:00  26851.277344  1817.534790  90.946075
2023-05-24 00:00:00+00:00  27139.914062  1849.520020  91.599770

[2022 rows x 3 columns]


/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



In [20]:
pf[(50, 53, 'ETH-USD')].plot().show()

In [10]:
symbols = ["BTC-USD", "ETH-USD", "ADA-USD"]
price = vbt.YFData.download(symbols, period='6mo', missing_index='drop').get('Close')
bbands = vbt.BBANDS.run(price)

def plot(index, bbands):
    bbands = bbands.loc[index]
    fig = vbt.make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.15,
        subplot_titles=('%B', 'Bandwidth'))
    fig.update_layout(template='vbt_dark', showlegend=False, width=750, height=400)
    bbands.percent_b.vbt.ts_heatmap(
        trace_kwargs=dict(zmin=0, zmid=0.5, zmax=1, colorscale='Spectral', colorbar=dict(
            y=(fig.layout.yaxis.domain[0] + fig.layout.yaxis.domain[1]) / 2, len=0.5
        )), add_trace_kwargs=dict(row=1, col=1), fig=fig)
    bbands.bandwidth.vbt.ts_heatmap(
        trace_kwargs=dict(colorbar=dict(
            y=(fig.layout.yaxis2.domain[0] + fig.layout.yaxis2.domain[1]) / 2, len=0.5
        )), add_trace_kwargs=dict(row=2, col=1), fig=fig)
    return fig

vbt.save_animation('bbands.gif', bbands.wrapper.index, plot, bbands, delta=90, step=3, fps=3)

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/vectorbt/data/base.py:692: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use eit

  0%|          | 0/31 [00:00<?, ?it/s]

/Users/shreyjoshi/dev/market-probe/venv/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
